<img src = "https://images2.imgbox.com/32/ac/wucGkuem_o.png" width="300">

In [1]:
import pandas as pd
import spacy

In [2]:
dados_treino = pd.read_csv("treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
41361,Ali foi um dos grandes frasistas do esporte; l...,Muhammad Ali não apenas foi um dos maiores ast...,2016-04-06,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/06/1...
51328,Vazio deixado pelo fechamento da rede 2001 não...,"O tempo corre rápido: há uns anos, ninguém pod...",2015-12-22,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/12...
72784,Hollande confirma desmonte de campo de refugia...,"François Hollande, presidente da França, refor...",2016-09-26,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/09/181...
13144,"Comércio paulista cresce, mas sinal de vida ai...","Depois da grande queimada, aparecem raros brot...",2016-02-09,colunas,viniciustorres,http://www1.folha.uol.com.br/colunas/viniciust...
79434,Homem esfaqueia policial em aeroporto de Michi...,"O Aeroporto Internacional Bishop, em Flint, no...",2017-06-21,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...


In [0]:
#Rodas apenas uma vez
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 89.9MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=e6686ba775d01478f4e0e215883920b63ad81fd819ef08daf6ceeaba8f527e86
  Stored in directory: /tmp/pip-ephem-wheel-cache-tsi63j7g/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [3]:
nlp = spacy.load("pt_core_news_sm")

In [4]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [5]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [6]:
type(doc[2])

spacy.tokens.token.Token

In [7]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [8]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [9]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = 3)]

In [11]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [13]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [14]:
w2v_modelo

In [15]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [16]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [18]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2022-03-26 20:21:21,020 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-03-26T20:21:21.020898', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-03-26 20:21:21,020 : - collecting all words and their counts
2022-03-26 20:21:21,021 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-26 20:21:21,028 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-03-26 20:21:21,035 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-03-26 20:21:21,042 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-03-26 20:21:21,049 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-03-26 20:21:21,056 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word

In [19]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [20]:
w2v_modelo.corpus_count

84466

In [21]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2022-03-26 20:21:27,659 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2022-03-26T20:21:27.659247', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2022-03-26 20:21:28,026 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:21:28,033 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:21:28,042 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:21:28,042 : - EPOCH - 1 : training on 540242 raw words (486037 effective words) took 0.4s, 1297520 effective words/s
2022-03-26 20:21:28,402 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:21:28,405 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:21:28,408 : - worker thr

2022-03-26 20:21:35,404 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:21:35,411 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:21:35,412 : - EPOCH - 21 : training on 540242 raw words (486079 effective words) took 0.4s, 1371593 effective words/s
2022-03-26 20:21:35,763 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:21:35,766 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:21:35,772 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:21:35,773 : - EPOCH - 22 : training on 540242 raw words (486067 effective words) took 0.4s, 1377225 effective words/s
2022-03-26 20:21:36,119 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:21:36,123 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:21:36,133 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:21:36,133 : - EPOCH - 23 : trainin

(14583694, 16207260)

In [22]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5782870650291443),
 ('facebook', 0.5041478872299194),
 ('waze', 0.49046817421913147),
 ('amazon', 0.478201687335968),
 ('airbnb', 0.46798449754714966),
 ('yahoo', 0.4502212405204773),
 ('uber', 0.4500688910484314),
 ('tesla', 0.446879506111145),
 ('software', 0.4447990357875824),
 ('volkswagen', 0.4419955909252167)]

In [23]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5780666470527649),
 ('amazon', 0.5668013095855713),
 ('sky', 0.513708770275116),
 ('viajante', 0.5103511214256287),
 ('sony', 0.507834792137146),
 ('tesla', 0.5046946406364441),
 ('linkedin', 0.49667441844940186),
 ('lego', 0.4914659857749939),
 ('walmart', 0.4873628318309784),
 ('buffett', 0.48505157232284546)]

In [24]:
w2v_modelo.wv.most_similar("barcelona")

[('barça', 0.5862477421760559),
 ('bayern', 0.5695069432258606),
 ('chelsea', 0.5559929609298706),
 ('madrid', 0.5494162440299988),
 ('lazio', 0.5354681015014648),
 ('united', 0.5348320603370667),
 ('juventus', 0.5225729942321777),
 ('munique', 0.5139992237091064),
 ('psg', 0.5087335109710693),
 ('leicester', 0.503046989440918)]

In [25]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.589506983757019),
 ('chuteiras', 0.5402089357376099),
 ('cristiano', 0.5383704900741577),
 ('cavani', 0.5088627934455872),
 ('barça', 0.5043180584907532),
 ('neymar', 0.489362895488739),
 ('ronaldo', 0.48808449506759644),
 ('barcelona', 0.4845430850982666),
 ('benzema', 0.48074859380722046),
 ('tevez', 0.4804903268814087)]

In [26]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6728121042251587),
 ('chrysler', 0.6474500298500061),
 ('embraer', 0.6380140781402588),
 ('braskem', 0.6307033896446228),
 ('csn', 0.6247340440750122),
 ('honda', 0.6204495429992676),
 ('tesla', 0.5832485556602478),
 ('fiat', 0.5808078646659851),
 ('metalúrgicos', 0.5762304067611694),
 ('toyota', 0.5755845308303833)]

In [29]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2022-03-26 20:22:47,220 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-03-26T20:22:47.219338', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-03-26 20:22:47,223 : - collecting all words and their counts
2022-03-26 20:22:47,223 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-26 20:22:47,229 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-03-26 20:22:47,236 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-03-26 20:22:47,242 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-03-26 20:22:47,250 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-03-26 20:22:47,257 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word

2022-03-26 20:22:56,265 : - EPOCH - 10 : training on 540242 raw words (486170 effective words) took 0.9s, 569511 effective words/s
2022-03-26 20:22:57,112 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:22:57,113 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:22:57,119 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:22:57,119 : - EPOCH - 11 : training on 540242 raw words (486098 effective words) took 0.8s, 575214 effective words/s
2022-03-26 20:22:57,961 : - worker thread finished; awaiting finish of 2 more threads
2022-03-26 20:22:57,965 : - worker thread finished; awaiting finish of 1 more threads
2022-03-26 20:22:57,976 : - worker thread finished; awaiting finish of 0 more threads
2022-03-26 20:22:57,976 : - EPOCH - 12 : training on 540242 raw words (486157 effective words) took 0.8s, 572627 effective words/s
2022-03-26 20:22:58,818 : - worker thread finished; awaiting finish of 2 more threads
2022-

(14584442, 16207260)

In [30]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.4387425482273102),
 ('android', 0.4097253978252411),
 ('waze', 0.38893333077430725),
 ('buffett', 0.3842429518699646),
 ('apple', 0.38127344846725464),
 ('anunciantes', 0.36959028244018555),
 ('volkswagen', 0.3694041073322296),
 ('yahoo', 0.3648589849472046),
 ('toshiba', 0.3571029305458069),
 ('walmart', 0.35219624638557434)]

In [31]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5782870650291443),
 ('facebook', 0.5041478872299194),
 ('waze', 0.49046817421913147),
 ('amazon', 0.478201687335968),
 ('airbnb', 0.46798449754714966),
 ('yahoo', 0.4502212405204773),
 ('uber', 0.4500688910484314),
 ('tesla', 0.446879506111145),
 ('software', 0.4447990357875824),
 ('volkswagen', 0.4419955909252167)]

In [32]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5567712783813477),
 ('motors', 0.507946252822876),
 ('audi', 0.47469547390937805),
 ('honda', 0.4586760997772217),
 ('bmw', 0.45789316296577454),
 ('autoguiados', 0.4520058333873749),
 ('montadora', 0.44724422693252563),
 ('airbags', 0.44605767726898193),
 ('taubaté', 0.4457864761352539),
 ('coletivas', 0.4446195662021637)]

In [33]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6728121042251587),
 ('chrysler', 0.6474500298500061),
 ('embraer', 0.6380140781402588),
 ('braskem', 0.6307033896446228),
 ('csn', 0.6247340440750122),
 ('honda', 0.6204495429992676),
 ('tesla', 0.5832485556602478),
 ('fiat', 0.5808078646659851),
 ('metalúrgicos', 0.5762304067611694),
 ('toyota', 0.5755845308303833)]

In [34]:
w2v_modelo.wv.save_word2vec_format("models/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("models/modelo_skipgram.txt", binary=False)

2022-03-26 20:23:13,031 : - storing 12924x300 projection weights into models/modelo_cbow.txt
2022-03-26 20:23:14,918 : - storing 12924x300 projection weights into models/modelo_skipgram.txt
